# Blood Cell Classification


In [ ]:
%pip install -r requirements.txt


In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [1]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, MaxPool2D, Flatten
from keras.preprocessing import image

2023-08-18 22:51:09.728618: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-18 22:51:09.754080: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-18 22:51:10.162521: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Pre-processing

In [2]:
generator = image.ImageDataGenerator(
        rescale = 1./255,
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)

In [3]:
dataset = generator.flow_from_directory(
    shuffle = True,
    batch_size = 32,
    target_size = (80, 80),
    class_mode = 'categorical',
    subset = 'training',
    directory = 'datasets/dataset2-master/dataset2-master/images/TRAIN'
)

Found 9957 images belonging to 4 classes.


In [4]:
dataset.image_shape

(80, 80, 3)

## Model

In [5]:
def model_builder():
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides = (1, 1), activation = 'relu', input_shape = (80, 80, 3)))
    model.add(Conv2D(80, (3,3), strides = (1, 1), activation = 'relu'))
    model.add(MaxPool2D(pool_size = (2,2)))
    model.add(Conv2D(64, (3,3), strides = (1,1), activation = 'relu'))
    model.add(Dropout(0.25))
    model.add(Flatten())

    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation = 'softmax'))


    model.compile(loss = 'categorical_crossentropy', optimizer = 'adadelta', metrics = ['accuracy'])
    model.build(dataset.image_shape)
    model.summary()
    
    return model

In [7]:
nn = model_builder()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 78, 78, 64)        1792      
                                                                 
 conv2d_4 (Conv2D)           (None, 76, 76, 80)        46160     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 38, 38, 80)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 36, 36, 64)        46144     
                                                                 
 dropout_2 (Dropout)         (None, 36, 36, 64)        0         
                                                                 
 flatten_1 (Flatten)         (None, 82944)             0         
                                                      

In [8]:
nn.fit(dataset, steps_per_epoch = None, epochs = 30, verbose = 1)
nn.save('Model.h5')

Epoch 1/30


2023-08-18 22:51:40.175345: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/dropout_2/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


: 

: 